[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/EMQA/blob/main/EMQA_vol_clustering/EMQA_vol_clustering.ipynb)

# EMQA_vol_clustering
Volatility clustering in energy returns -- visual evidence from Brent crude oil.
**Output:** `vol_clustering.pdf`

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.facecolor': 'none',
    'axes.facecolor': 'none',
    'savefig.facecolor': 'none',
    'savefig.transparent': True,
    'axes.grid': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
    'figure.figsize': (12, 6),
})

COLORS = {
    'blue': '#1A3A6E', 'red': '#CD0000', 'green': '#2E7D32',
    'orange': '#E67E22', 'purple': '#8E44AD', 'gray': '#808080',
    'cyan': '#00BCD4', 'amber': '#B5853F'
}

def save_fig(fig, name):
    fig.savefig(name, bbox_inches='tight', transparent=True, dpi=300)
    print(f"Saved: {name}")


In [ ]:
import yfinance as yf

def fetch(ticker, start='2020-01-01', end='2025-12-31'):
    d = yf.download(ticker, start=start, end=end, progress=False)
    if isinstance(d.columns, pd.MultiIndex):
        return d['Close'].squeeze().dropna()
    return d['Close'].dropna()


In [ ]:
# Fetch Brent crude
brent = fetch('BZ=F', start='2018-01-01')
log_ret = np.log(brent / brent.shift(1)).dropna()
print(f"Brent returns: {len(log_ret)} observations, {log_ret.index[0].date()} to {log_ret.index[-1].date()}")

In [ ]:
# 3-panel: Returns, Squared returns, Rolling volatility
fig, axes = plt.subplots(3, 1, figsize=(12, 12), sharex=True)

# Panel 1 - Returns
axes[0].plot(log_ret.index, log_ret.values, color=COLORS['blue'], linewidth=0.5)
axes[0].set_ylabel('Log Returns')
axes[0].set_title('Brent Crude Oil - Volatility Clustering')

# Panel 2 - Squared returns
axes[1].plot(log_ret.index, log_ret.values**2, color=COLORS['red'], linewidth=0.5)
axes[1].set_ylabel('Squared Returns')

# Panel 3 - Rolling volatility (30-day, annualized)
roll_vol = log_ret.rolling(30).std() * np.sqrt(252)
axes[2].plot(roll_vol.index, roll_vol.values, color=COLORS['green'], linewidth=1)
axes[2].set_ylabel('Annualized Volatility')
axes[2].set_xlabel('Date')

# Highlight COVID (Mar-Jun 2020) and Ukraine (Feb-Jun 2022)
for ax in axes:
    ax.axvspan(pd.Timestamp('2020-03-01'), pd.Timestamp('2020-06-30'),
               alpha=0.15, color=COLORS['orange'], label='COVID-19')
    ax.axvspan(pd.Timestamp('2022-02-01'), pd.Timestamp('2022-06-30'),
               alpha=0.15, color=COLORS['purple'], label='Ukraine conflict')

# Legend on bottom axis only
handles, labels = axes[2].get_legend_handles_labels()
by_label = dict(zip(labels, handles))
axes[2].legend(by_label.values(), by_label.keys(),
               loc='upper center', bbox_to_anchor=(0.5, -0.10), frameon=False, ncol=2)

plt.tight_layout()
save_fig(fig, 'vol_clustering.pdf')
plt.show()